## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [1]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict
from nltk.tokenize import word_tokenize

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [19]:

pip install --upgrade nltk

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/georgegarcia/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/georgegarcia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [8]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute(
                            '''CREATE TABLE IF NOT EXISTS conventions (
                              text TEXT,
                              party TEXT
                            )
                            ''')
# Function for cleaning and tokenizing text
def process_text(text):
    tokens = word_tokenize(text)
    cleaned_text = " ".join(tokens)
    return cleaned_text

# Example text and party
text = "This is some example text."
party = "Democratic"

# Clean and tokenize text
cleaned_text = process_text(text)

# Insert the cleaned and tokenized text and party into the database
convention_cur.execute("""
INSERT INTO conventions (text, party)
VALUES (?, ?)
""", (cleaned_text, party))

# Commit the changes to the database
convention_db.commit()

convention_data = []
query_results = convention_cur.execute("SELECT text, party FROM conventions")
for row in query_results :
    convention_data.append([row[0], row[1]])# store the results in convention_data
    pass # remove this
    


Let's look at some random entries and see if they look right. 

In [9]:
random.choices(convention_data,k=10)

[['Virginia.', 'Democratic'],
 ['My journey to a religious life was not a traditional route if there is such a thing. In 1978 as a medical school student at Georgetown University, I joined the army to help pay for my tuition and ended up devoting 29 years to the military, serving as a doctor and a surgeon in places like Afghanistan and Egypt’s Sinai Peninsula. After much prayer and contemplation I entered my religious order in 2002, working to serve the poor and the sick in Haiti, Sudan, Kenya, Iraq and in Washington, D.C. Humility is at the foundation of our order which makes it very difficult to talk about myself but I can speak about my experience working for those fleeing, war-torn and impoverished countries all around the world. Those refugees all share a common experience. They have been all marginalized, viewed as insignificant, powerless and voiceless. And while we tend to think of the marginalized as living beyond our borders, the truth is the largest marginalized group in the

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [92]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2891 as features in the model.


In [116]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here
    tokens = set(word_tokenize(text))
    dict = {word: True for word in tokens if word in fw}
    return dict
    ret_dict = dict()
    

In [117]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

AssertionError: 

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [118]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [120]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [121]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.436


In [122]:
classifier.show_most_informative_features(25)

Most Informative Features
                 radical = True           Republ : Democr =     37.2 : 1.0
                   media = True           Republ : Democr =     36.1 : 1.0
             enforcement = True           Republ : Democr =     33.0 : 1.0
                   votes = True           Democr : Republ =     24.0 : 1.0
               greatness = True           Republ : Democr =     19.4 : 1.0
                   China = True           Republ : Democr =     18.6 : 1.0
                   taxes = True           Republ : Democr =     16.7 : 1.0
                 destroy = True           Republ : Democr =     16.2 : 1.0
                 climate = True           Democr : Republ =     15.5 : 1.0
                supports = True           Republ : Democr =     15.2 : 1.0
                 company = True           Republ : Democr =     13.1 : 1.0
                    Mike = True           Republ : Democr =     12.9 : 1.0
                    Lady = True           Republ : Democr =     12.3 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### The things that stood out to me was that word radical was used more by republican than democrats, the word slady was used more by republican. I was woundering how much was this influenced by donald trump 

_Your observations to come._



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [73]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [74]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [82]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.
query_results = cong_cur.execute(
                            '''
                            CREATE TABLE IF NOT EXISTS results (
                              tweet_text TEXT,
                              party TEXT
                            )
                            ''')

def process_text(text):
    tokens = word_tokenize(tweet_text)
    cleaned_text = " ".join(tokens)
    return cleaned_text

# Example tweet and party
tweet_text = "This is some example text."
party = "Democratic"

# Clean and tokenize text
cleaned_text = process_text(tweet_text)

# Insert the cleaned and tokenized text and party into the database
cong_cur.execute("""
INSERT INTO results (tweet_text, party)
VALUES (?, ?)
""", (cleaned_text, party))

# Commit the changes to the database
cong_db.commit()

teet_data = []
query_results = cong_cur.execute("SELECT tweet_text, party FROM results")
for row in query_results :
    tweet_data.append([row[0], row[1]])# store the results in convention_data
    pass # remove this

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [83]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [85]:
tweet_data_sample

[['This is some example text .', 'Democratic'],
 ['This is some example text .', 'Democratic'],
 ['This is some example text .', 'Democratic'],
 ['This is some example text .', 'Democratic'],
 ['This is some example text .', 'Democratic'],
 ['This is some example text .', 'Democratic'],
 ['This is some example text .', 'Democratic'],
 ['This is some example text .', 'Democratic'],
 ['This is some example text .', 'Democratic'],
 ['This is some example text .', 'Democratic']]

In [128]:

for tweet, party in tweet_data_sample :
    estimated_party = classifier.classify(features)
    results[party][estimated_party] += 1
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: This is some example text .
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: This is some example text .
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: This is some example text .
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: This is some example text .
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: This is some example text .
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: This is some example text .
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: This is some example text .
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: This is some example text .
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: This is some example

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [129]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
    features = conv_features(tweet, feature_words)
    for p in parties:
        if classifier.classify(features) == p:
            results[party][p] += 1
    if idx == num_to_score:
        break
   
    # get the estimated party
    estimated_party = classifier.classify(features)
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [130]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 0, 'Democratic': 0}),
             'Democratic': defaultdict(int,
                         {'Republican': 0, 'Democratic': 28})})

### Reflections

_the cod eiwrote is only looking at the democratic party and is able to predict it pretty well but it is missing the republican party, it might have to di with my example text not having rthe republican_ 